# Integrating Flotorch Session Management with CrewAI

This notebook demonstrates how to integrate Flotorch's session management with a CrewAI agent. By using `FlotorchCrewAISession` as the storage backend for CrewAI's `ShortTermMemory`, we can create a conversational agent that remembers the context of the current session.

### Prerequesit
Configure model, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Concepts:
- **`FlotorchCrewAILLM`**: The language model powering the agent.
- **`FlotorchCrewAISession`**: The session storage mechanism for persisting conversation history.
- **`ShortTermMemory`**: CrewAI's memory system, configured to use the Flotorch session backend.
- **`Agent`, `Task`, `Crew`**: The core components for building and orchestrating the AI workflow.

## 1. Setup and Configuration

The following cells install the necessary packages, configure API credentials, and import all required components from the Flotorch and CrewAI libraries.

In [ ]:
# install flotorch adk package
%pip install flotorch[crewai]

In [ ]:
%pip install crewai crewai_tools crewai_tools[mcp]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import Necessary libraries
from flotorch.crewai.llm import FlotorchCrewAILLM
from flotorch.crewai.sessions import FlotorchCrewAISession
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Agent, Task, Crew

print("Imported necessary libraries successfully")

## 2. Model Configuration

Here, we initialize the `FlotorchCrewAILLM` with our model configuration. This will serve as the reasoning engine for our CrewAI agent.

In [ ]:
model  =  FlotorchCrewAILLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured: {model.model}")

## 3. Session Storage Setup

We configure Flotorch session storage and integrate it with CrewAI's short-term memory system. This enables the agent to maintain conversational context.

In [ ]:
short_term_storage = FlotorchCrewAISession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

short_term_memory = ShortTermMemory(storage = short_term_storage)

print("short_term_memory initialized")

## 4. Agent and Task Configuration

Next, we define the agent's persona (`role`, `goal`, `backstory`) and the specific `task` it must perform. The agent is designed for natural, context-aware conversation.

In [ ]:
# Create the Agent
agent = Agent(
    role="Conversational Memory Agent",
    goal=""" You are a helpful conversational agent with short-term memory.
    You remember recent conversation context to give better answers.
    You respond naturally to user queries while recalling relevant past information.
    user_query: {query}""",
    backstory=(
        "An engaging assistant who remembers recent interactions "
        "and uses them to answer questions accurately and conversationally."
    ),
    allow_delegation=False,
    verbose=False,
    llm=model
)

# Create the Task (goal is same as agent)
task = Task(
    description=""" You are a helpful conversational agent with short-term memory.
    You remember recent conversation context to give better answers.
    You respond naturally to user queries while recalling relevant past information.
    user_query: {query}""",
    expected_output=(
        "A natural, conversational answer that may use recent memory if relevant."
    ),
    agent=agent
)
# Create custom tools and agent
print(f"Agent and Task created successfully: {agent.role}")

## 5. Crew Assembly

We create a `Crew` to orchestrate the agent and its task. Crucially, we equip the crew with the `short_term_memory` module configured in the previous step.

In [ ]:
crew = Crew(
    agents = [agent],
    tasks = [task],
    short_term_memory = short_term_memory,
    verbose = False
)

print("Crew created with agent,task and short_term memory")


## 6. Interactive Chat

This loop starts an interactive chat session. The agent will use the Flotorch-backed session memory to recall context from previous messages within this conversation. Type 'exit' to end the session.

In [ ]:
while True:

    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    response = crew.kickoff(inputs = {"query":user_query})
    print(f"Assistant:{response.raw}")

## Summary

This notebook successfully demonstrated how to create a conversational agent with short-term memory.  
By integrating **Flotorch's session management** with **CrewAI**, we achieved a **stateful interaction** where the agent can recall information from earlier in the conversation.

### Key Achievements

- **Session Persistence**  
  Configured `FlotorchCrewAISession` as the backend for CrewAI's `ShortTermMemory`, enabling the agent to maintain context across turns.

- **Context-Aware Responses**  
  The agent successfully referenced previously mentioned details (e.g., the user's name) to provide more natural and relevant replies.

- **Streamlined Integration**  
  Demonstrated a clear and effective method to connect Flotorch's robust session infrastructure with CrewAI’s powerful agent framework.
